# <b>Подготовка</b>

## Начало

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('vacancies_2024.csv')
df

C:\Users\PC-1\AppData\Local\Temp\ipykernel_8616\17425615.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('vacancies_2024.csv')


,name,key_skills,salary_from,salary_to,salary_currency,area_name,published_at
0,SAP R/3 functional consultant,NaN,NaN,NaN,NaN,Москва,2003-01-24T21:30:49+0300
1,Broadcast encryption engineer,NaN,NaN,2000.0,USD,Южная Корея,2003-01-31T18:24:11+0300
2,Embedded OS engineering manager,NaN,NaN,2000.0,USD,Москва,2003-01-31T18:25:43+0300
3,Coordinator on Intellectual Property Projects,NaN,NaN,NaN,NaN,Москва,2003-02-12T10:40:50+0300
4,Outsourcing Projects Coordinator,NaN,NaN,NaN,NaN,Москва,2003-02-14T16:31:06+0300
...,...,...,...,...,...,...,...
6915292,Дизайнер UX/UI,Adobe Photoshop\nAdobe Illustrator\nПодготовка...,130000.0,NaN,RUR,Москва,2024-11-30T12:32:11+0300
6915293,Руководитель по развитию перспективных проекто...,NaN,NaN,NaN,NaN,Москва,2024-11-30T12:29:40+0300
6915294,Tech Lead Node.js (Backend разработчик/руковод...,Git\nNode.js\nTypeScript\nJavaScript\nPostgreS...,250000.0,NaN,RUR,Санкт-Петербург,2024-11-30T22:29:02+0300
6915295,Специалист голосовой поддержки водителей со зн...,Телефонные переговоры\nРабота с большим объемо...,NaN,NaN,NaN,Джизак,2024-11-30T10:07:58+0300


## Курс валют 

In [17]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta


BASE_URL = "http://www.cbr.ru/scripts/XML_daily.asp"

salary_currencies = {
    "BYR": "R01090",
    "USD": "R01235",
    "EUR": "R01239",
    "KZT": "R01335",
    "UAH": "R01720",
    "AZN": "R01020A",
    "KGS": "R01370",
    "UZS": "R01717",
    "GEL": "R01210",
}

start_date = datetime(2003, 1, 1)
end_date = datetime(2024, 11, 1)

current_date = start_date
dates = []
while current_date <= end_date:
    dates.append(current_date.strftime("%d/%m/%Y"))
    current_date += timedelta(days=32)
    current_date = current_date.replace(day=1)


def get_currency_rate(date):
    params = {"date_req": date}
    response = requests.get(BASE_URL, params=params)

    root = ET.fromstring(response.content)
    rowForDf = {"date": date}
    
    for valute in root.findall("Valute"):
        char_code = valute.find("CharCode").text
        vunitRate = valute.find("VunitRate").text 

        if char_code in salary_currencies.keys(): 
            rowForDf[char_code] = vunitRate.replace(",", ".")
            
    return rowForDf


all_data = []
for date in dates:
    rowForDf = get_currency_rate(date)
    all_data.append(rowForDf)

columns = ["date", "BYR", "USD", "EUR", "KZT", "UAH", "AZN", "KGS", "UZS", "GEL"]
currency_df = pd.DataFrame(all_data, columns=columns)

currency_df['date'] = pd.to_datetime(currency_df['date'], format="%d/%m/%Y").dt.strftime("%Y-%m")
currency_df.to_csv("currency.csv", index=False, encoding="utf-8")


## Фильтрация вакансий

In [18]:
#web developer ключевые слова:
keywords = [
    'web develop', 'веб разработчик', 'web разработчик', 'web programmer', 'web программист', 'веб программист', 
    'битрикс разработчик', 'bitrix разработчик', 'битрикс девелопер', 'drupal разработчик', 'cms разработчик', 
    'wordpress разработчик', 'wp разработчик', 'joomla', 'drupal developer', 'cms developer', 'wordpress developer', 
    'wp developer', 'joomla developer', 'frontend developer', 'backend developer', 'fullstack developer', 
    'фронтенд разработчик', 'бекенд разработчик', 'фулстек разработчик', 'frontend engineer', 'backend engineer', 
    'fullstack engineer', 'front-end developer', 'back-end developer', 'full-stack developer', 'front-end engineer', 
    'back-end engineer', 'full-stack engineer', 'веб инженер', 'site developer', 'сайт разработчик', 
    'инженер-программист', 'разработчик сайтов', 'developer web', 'web engineer', 
    'php developer', 'php разработчик', 'python веб разработчик', 'node.js developer', 'node.js разработчик', 
    'javascript developer', 'javascript разработчик', 'typescript developer', 'typescript разработчик', 
    'vue.js developer', 'vue.js разработчик', 'react developer', 'react разработчик', 
    'angular developer', 'angular разработчик', 'django developer', 'django разработчик', 
    'джанго разработчик', 'flask developer', 'flask разработчик', 'ларевел разработчик', 
    'laravel developer', 'laravel разработчик', 'symfony developer', 'symfony разработчик', 
    'саймфони разработчик', 'ruby on rails developer', 'ruby разработчик', 'rails developer', 
    'ror developer', 'ruby on rails разработчик', 'рубивонрейлс разработчик', 'react.js разработчик', 
    'react.js developer', 'next.js developer', 'next.js разработчик'
]

keywords_pattern = '|'.join(keywords)

In [19]:
filtred_df = df[df['name'].str.contains(keywords_pattern, case=False)]
filtred_df

,name,key_skills,salary_from,salary_to,salary_currency,area_name,published_at
168,Web Developer,NaN,NaN,NaN,NaN,Санкт-Петербург,2003-08-26T00:00:00+0400
346,Инженер-программист по автоматизации тестирования,NaN,NaN,NaN,NaN,Москва,2003-09-10T12:31:42+0400
565,Инженер-программист по внедрению и настройке П...,NaN,NaN,800.0,USD,Москва,2003-09-25T13:30:38+0400
586,Инженер-программист,NaN,NaN,700.0,USD,Москва,2003-09-29T00:00:00+0400
640,Инженер-программист по внедрению систем матери...,NaN,NaN,NaN,NaN,Москва,2003-10-01T16:26:58+0400
...,...,...,...,...,...,...,...
6915140,React Native / React.js developer,React Native\nReact\nАдаптивная верстка\nАнгли...,1500.0,3000.0,USD,Минск,2024-11-29T22:31:51+0300
6915200,WordPress разработчик,Wordpress\nFigma,NaN,NaN,NaN,Ташкент,2024-11-30T15:23:30+0300
6915220,Middle Full-stack Developer,Python\nGit\nDocker\nLinux\nPostgreSQL\nDjango...,NaN,NaN,NaN,Актобе,2024-11-30T12:41:24+0300
6915224,"Ведущий инженер-программист (Senior C++/Qt, Об...",NaN,NaN,NaN,NaN,Санкт-Петербург,2024-11-30T19:53:58+0300


## Перевод деняг

In [20]:
from datetime import datetime

df_currency = pd.read_csv('currency.csv', index_col='date')


def convert_to_rur(salary, salary_currency, published_at):
    if pd.isna(salary): 
        return None
    if salary_currency == 'RUR':
        return salary
    
    date = datetime.fromisoformat(published_at)
    date = date.strftime("%Y-%m")

    rate_that_day = df_currency.loc[date]
    
    if salary_currency in df_currency.columns:
        return salary * rate_that_day[salary_currency]


def get_avg_salary(salary_from, salary_to):
    if pd.isna(salary_from) and pd.isna(salary_to):
        return None
    elif pd.isna(salary_from):
        return salary_to
    elif pd.isna(salary_to):
        return salary_from
    else:
        return (salary_from + salary_to) / 2
    

filtred_df['salary'] = filtred_df.apply(lambda row: get_avg_salary(row['salary_from'], row['salary_to']), axis=1)
filtred_df['salary'] = filtred_df.apply(lambda row: convert_to_rur(row['salary'], row['salary_currency'], row['published_at']), axis=1)
# csv_merged = csv_merged.dropna(subset=['salary'])
csv_merged = filtred_df[['name', 'key_skills','salary', 'area_name', 'published_at']]

csv_merged

C:\Users\PC-1\AppData\Local\Temp\ipykernel_8616\3808020099.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtred_df['salary'] = filtred_df.apply(lambda row: get_avg_salary(row['salary_from'], row['salary_to']), axis=1)
C:\Users\PC-1\AppData\Local\Temp\ipykernel_8616\3808020099.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtred_df['salary'] = filtred_df.apply(lambda row: convert_to_rur(row['salary'], row['salary_currency'], row['published_at']), axis=1)


,name,key_skills,salary,area_name,published_at
168,Web Developer,NaN,NaN,Санкт-Петербург,2003-08-26T00:00:00+0400
346,Инженер-программист по автоматизации тестирования,NaN,NaN,Москва,2003-09-10T12:31:42+0400
565,Инженер-программист по внедрению и настройке П...,NaN,24402.88,Москва,2003-09-25T13:30:38+0400
586,Инженер-программист,NaN,21352.52,Москва,2003-09-29T00:00:00+0400
640,Инженер-программист по внедрению систем матери...,NaN,NaN,Москва,2003-10-01T16:26:58+0400
...,...,...,...,...,...
6915140,React Native / React.js developer,React Native\nReact\nАдаптивная верстка\nАнгли...,218300.85,Минск,2024-11-29T22:31:51+0300
6915200,WordPress разработчик,Wordpress\nFigma,NaN,Ташкент,2024-11-30T15:23:30+0300
6915220,Middle Full-stack Developer,Python\nGit\nDocker\nLinux\nPostgreSQL\nDjango...,NaN,Актобе,2024-11-30T12:41:24+0300
6915224,"Ведущий инженер-программист (Senior C++/Qt, Об...",NaN,NaN,Санкт-Петербург,2024-11-30T19:53:58+0300


## Еще фильтрации(-ия)

In [23]:
filtred_df = csv_merged[csv_merged['salary'] <= 10_000_000]
filtred_df

,name,key_skills,salary,area_name,published_at
565,Инженер-программист по внедрению и настройке П...,NaN,24402.88,Москва,2003-09-25T13:30:38+0400
586,Инженер-программист,NaN,21352.52,Москва,2003-09-29T00:00:00+0400
698,Инженер-программист OS/390,NaN,61228.40,Москва,2003-10-05T13:03:37+0400
700,Веб программист (Java),NaN,45921.30,Москва,2003-10-06T00:00:00+0400
831,Инженер-программист по автоматизации тестирования,NaN,15307.10,Москва,2003-10-10T11:53:19+0400
...,...,...,...,...,...
6915081,Ведущий инженер-программист / руководитель гру...,NaN,152000.00,Приморск (Ленинградская область),2024-11-29T15:58:00+0300
6915083,Ведущий инженер-программист / руководитель гру...,NaN,152000.00,Всеволожск,2024-11-29T15:58:00+0300
6915093,Инженер-программист,NaN,150000.00,Ижевск,2024-11-29T10:19:33+0300
6915108,Инженер-программист,NaN,100000.00,Екатеринбург,2024-11-29T13:50:36+0300


## Сохранение в отдельный файл

In [24]:
filtred_df.to_csv("filtred_df.csv", index=False, encoding="utf-8")

# <b>Страница «Общая статистика»</b>

## Динамика уровня зарплат по годам

## Динамика количества вакансий по годам

## Уровень зарплат по городам

## Доля вакансий по городам

## ТОП-20 навыков по годам

# <b>Страница «Востребованность»</b>